In [103]:
import pandas as pd
import numpy as np

In [104]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/ericiortega/ids705-phishing-detection/refs/heads/main/data/raw/trec_07.csv"
)
df

,sender,receiver,date,subject,body,label,urls
0,Tomas Jacobs <RickyAmes@aol.com>,the00@speedy.uwaterloo.ca,"Sun, 08 Apr 2007 21:00:48 +0300","Generic Cialis, branded quality@",\n\n\n\n\n\n\nDo you feel the pressure to perf...,1,0
1,Yan Morin <yan.morin@savoirfairelinux.com>,debian-mirrors@lists.debian.org,"Sun, 08 Apr 2007 12:52:30 -0400",Typo in /debian/README,"Hi, i've just updated from the gulus and I che...",0,1
2,Sheila Crenshaw <7stocknews@tractionmarketing....,the00@plg.uwaterloo.ca,"Sun, 08 Apr 2007 17:12:19 +0000",authentic viagra,Mega authenticV I A G R A $ DISCOUNT priceC...,1,1
3,Stormy Dempsey <vqucsmdfgvsg@ruraltek.com>,opt4@speedy.uwaterloo.ca,"Sun, 08 Apr 2007 17:15:47 -0100",Nice talking with ya,"\nHey Billy, \n\nit was really fun going out t...",1,1
4,"""Christi T. Jernigan"" <dcube@totalink.net>",ktwarwic@speedy.uwaterloo.ca,"Sun, 08 Apr 2007 19:19:07 +0200",or trembling; stomach cramps; trouble in sleep...,"\nsystem"" of the home. It will have the capab...",1,0
...,...,...,...,...,...,...,...
53752,SCC <Gerry.Rossi4360@kinki-kids.com>,Deficient <deficient@flax9.uwaterloo.ca>,"Fri, 06 Jul 2007 06:53:36 -0400",Job: just for you.,\n\n\n\nWhile we may have high ...,1,1
53753,Sydney Car Centre <Merrill8783@168city.com>,Gnitpick <gnitpick@flax9.uwaterloo.ca>,"Fri, 06 Jul 2007 06:59:51 -0400",the reply for your request for a job place [le...,\n\n\n\nWhile we may have high ...,1,1
53754,Philippe Grosjean <phgrosjean@sciviews.org>,Duncan Murdoch <murdoch@stats.uwo.ca>,"Fri, 06 Jul 2007 12:57:17 +0200","Re: [R] Me again, about the horrible documenta...","For those who are interested, I just cook a li...",0,1
53755,Bernhard Wellhöfer <Bernhard.Wellhoefer@gaia-g...,r-help@stat.math.ethz.ch,"Fri, 06 Jul 2007 12:43:12 +0200",Re: [R] RODBC problem,"Hello,\n\nas I wrote I call\n\n sqlFetch(chan...",0,1


In [105]:
print(df["date"].head())  # Check if dates were converted properly
print(df["date"].dtype)  # Check the data type
df["date"] = pd.to_datetime(df["date"], utc=True, errors="coerce")
print(df["date"].dtype)  # Now it should show 'datetime64[ns, UTC]'

0    Sun, 08 Apr 2007 21:00:48 +0300
1    Sun, 08 Apr 2007 12:52:30 -0400
2    Sun, 08 Apr 2007 17:12:19 +0000
3    Sun, 08 Apr 2007 17:15:47 -0100
4    Sun, 08 Apr 2007 19:19:07 +0200
Name: date, dtype: object
object
datetime64[ns, UTC]


In [106]:
df["text"] = df["subject"].fillna("") + " " + df["body"].fillna("")

In [107]:
# Ensure date conversion (handles different formats)
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Check data type after conversion
print("After conversion:", df["date"].dtype)

# Extracting components
df["day_of_week"] = df["date"].dt.day_name()  # Full weekday name
df["day"] = df["date"].dt.day  # Day of the month
df["month"] = df["date"].dt.month_name()  # Full month name
df["year"] = df["date"].dt.year  # Year
df["time"] = df["date"].dt.strftime("%H:%M:%S")  # Extract Time
df["timezone"] = df["date"].dt.tz  # Timezone info (if present)

After conversion: datetime64[ns, UTC]


In [108]:
df

,sender,receiver,date,subject,body,label,urls,text,day_of_week,day,month,year,time,timezone
0,Tomas Jacobs <RickyAmes@aol.com>,the00@speedy.uwaterloo.ca,2007-04-08 18:00:48+00:00,"Generic Cialis, branded quality@",\n\n\n\n\n\n\nDo you feel the pressure to perf...,1,0,"Generic Cialis, branded quality@ \n\n\n\n\n\n...",Sunday,8.0,April,2007.0,18:00:48,UTC
1,Yan Morin <yan.morin@savoirfairelinux.com>,debian-mirrors@lists.debian.org,2007-04-08 16:52:30+00:00,Typo in /debian/README,"Hi, i've just updated from the gulus and I che...",0,1,"Typo in /debian/README Hi, i've just updated f...",Sunday,8.0,April,2007.0,16:52:30,UTC
2,Sheila Crenshaw <7stocknews@tractionmarketing....,the00@plg.uwaterloo.ca,2007-04-08 17:12:19+00:00,authentic viagra,Mega authenticV I A G R A $ DISCOUNT priceC...,1,1,authentic viagra Mega authenticV I A G R A ...,Sunday,8.0,April,2007.0,17:12:19,UTC
3,Stormy Dempsey <vqucsmdfgvsg@ruraltek.com>,opt4@speedy.uwaterloo.ca,2007-04-08 18:15:47+00:00,Nice talking with ya,"\nHey Billy, \n\nit was really fun going out t...",1,1,"Nice talking with ya \nHey Billy, \n\nit was r...",Sunday,8.0,April,2007.0,18:15:47,UTC
4,"""Christi T. Jernigan"" <dcube@totalink.net>",ktwarwic@speedy.uwaterloo.ca,2007-04-08 17:19:07+00:00,or trembling; stomach cramps; trouble in sleep...,"\nsystem"" of the home. It will have the capab...",1,0,or trembling; stomach cramps; trouble in sleep...,Sunday,8.0,April,2007.0,17:19:07,UTC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53752,SCC <Gerry.Rossi4360@kinki-kids.com>,Deficient <deficient@flax9.uwaterloo.ca>,2007-07-06 10:53:36+00:00,Job: just for you.,\n\n\n\nWhile we may have high ...,1,1,Job: just for you. \n\n\n\nWhile we may ...,Friday,6.0,July,2007.0,10:53:36,UTC
53753,Sydney Car Centre <Merrill8783@168city.com>,Gnitpick <gnitpick@flax9.uwaterloo.ca>,2007-07-06 10:59:51+00:00,the reply for your request for a job place [le...,\n\n\n\nWhile we may have high ...,1,1,the reply for your request for a job place [le...,Friday,6.0,July,2007.0,10:59:51,UTC
53754,Philippe Grosjean <phgrosjean@sciviews.org>,Duncan Murdoch <murdoch@stats.uwo.ca>,2007-07-06 10:57:17+00:00,"Re: [R] Me again, about the horrible documenta...","For those who are interested, I just cook a li...",0,1,"Re: [R] Me again, about the horrible documenta...",Friday,6.0,July,2007.0,10:57:17,UTC
53755,Bernhard Wellhöfer <Bernhard.Wellhoefer@gaia-g...,r-help@stat.math.ethz.ch,2007-07-06 10:43:12+00:00,Re: [R] RODBC problem,"Hello,\n\nas I wrote I call\n\n sqlFetch(chan...",0,1,"Re: [R] RODBC problem Hello,\n\nas I wrote I c...",Friday,6.0,July,2007.0,10:43:12,UTC


In [109]:
# Count the number of records per year
year_counts = df["year"].value_counts().sort_index()

# Convert to DataFrame for display
year_counts_df = year_counts.reset_index()
year_counts_df.columns = ["year", "count"]
year_counts_df

,year,count
0,1900.0,1
1,1970.0,3
2,1980.0,5
3,1987.0,6
4,1990.0,38
5,1994.0,2
6,1997.0,1
7,1998.0,3
8,1999.0,5
9,2000.0,35


In [110]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# This ensures resources are loaded
nltk.download("stopwords")
nltk.download("wordnet")


# Preprocessing function
def preprocess_text(text):
    text = re.sub(r"<.*?>", "", str(text))  # Remove HTML
    tokens = text.split()
    return " ".join(tokens)


# Apply preprocessing
df["clean_text"] = df["subject"].fillna("") + " " + df["body"].fillna("")
df["clean_text"] = df["clean_text"].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tusunaiturumbekova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tusunaiturumbekova/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [111]:
df["clean_text"]

0        Generic Cialis, branded quality@ Do you feel t...
1        Typo in /debian/README Hi, i've just updated f...
2        authentic viagra Mega authenticV I A G R A $ D...
3        Nice talking with ya Hey Billy, it was really ...
4        or trembling; stomach cramps; trouble in sleep...
                               ...                        
53752    Job: just for you. While we may have high expe...
53753    the reply for your request for a job place [le...
53754    Re: [R] Me again, about the horrible documenta...
53755    Re: [R] RODBC problem Hello, as I wrote I call...
53756    I wanted the desk at his own laws: of the. But...
Name: clean_text, Length: 53757, dtype: object

In [112]:
df

,sender,receiver,date,subject,body,label,urls,text,day_of_week,day,month,year,time,timezone,clean_text
0,Tomas Jacobs <RickyAmes@aol.com>,the00@speedy.uwaterloo.ca,2007-04-08 18:00:48+00:00,"Generic Cialis, branded quality@",\n\n\n\n\n\n\nDo you feel the pressure to perf...,1,0,"Generic Cialis, branded quality@ \n\n\n\n\n\n...",Sunday,8.0,April,2007.0,18:00:48,UTC,"Generic Cialis, branded quality@ Do you feel t..."
1,Yan Morin <yan.morin@savoirfairelinux.com>,debian-mirrors@lists.debian.org,2007-04-08 16:52:30+00:00,Typo in /debian/README,"Hi, i've just updated from the gulus and I che...",0,1,"Typo in /debian/README Hi, i've just updated f...",Sunday,8.0,April,2007.0,16:52:30,UTC,"Typo in /debian/README Hi, i've just updated f..."
2,Sheila Crenshaw <7stocknews@tractionmarketing....,the00@plg.uwaterloo.ca,2007-04-08 17:12:19+00:00,authentic viagra,Mega authenticV I A G R A $ DISCOUNT priceC...,1,1,authentic viagra Mega authenticV I A G R A ...,Sunday,8.0,April,2007.0,17:12:19,UTC,authentic viagra Mega authenticV I A G R A $ D...
3,Stormy Dempsey <vqucsmdfgvsg@ruraltek.com>,opt4@speedy.uwaterloo.ca,2007-04-08 18:15:47+00:00,Nice talking with ya,"\nHey Billy, \n\nit was really fun going out t...",1,1,"Nice talking with ya \nHey Billy, \n\nit was r...",Sunday,8.0,April,2007.0,18:15:47,UTC,"Nice talking with ya Hey Billy, it was really ..."
4,"""Christi T. Jernigan"" <dcube@totalink.net>",ktwarwic@speedy.uwaterloo.ca,2007-04-08 17:19:07+00:00,or trembling; stomach cramps; trouble in sleep...,"\nsystem"" of the home. It will have the capab...",1,0,or trembling; stomach cramps; trouble in sleep...,Sunday,8.0,April,2007.0,17:19:07,UTC,or trembling; stomach cramps; trouble in sleep...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53752,SCC <Gerry.Rossi4360@kinki-kids.com>,Deficient <deficient@flax9.uwaterloo.ca>,2007-07-06 10:53:36+00:00,Job: just for you.,\n\n\n\nWhile we may have high ...,1,1,Job: just for you. \n\n\n\nWhile we may ...,Friday,6.0,July,2007.0,10:53:36,UTC,Job: just for you. While we may have high expe...
53753,Sydney Car Centre <Merrill8783@168city.com>,Gnitpick <gnitpick@flax9.uwaterloo.ca>,2007-07-06 10:59:51+00:00,the reply for your request for a job place [le...,\n\n\n\nWhile we may have high ...,1,1,the reply for your request for a job place [le...,Friday,6.0,July,2007.0,10:59:51,UTC,the reply for your request for a job place [le...
53754,Philippe Grosjean <phgrosjean@sciviews.org>,Duncan Murdoch <murdoch@stats.uwo.ca>,2007-07-06 10:57:17+00:00,"Re: [R] Me again, about the horrible documenta...","For those who are interested, I just cook a li...",0,1,"Re: [R] Me again, about the horrible documenta...",Friday,6.0,July,2007.0,10:57:17,UTC,"Re: [R] Me again, about the horrible documenta..."
53755,Bernhard Wellhöfer <Bernhard.Wellhoefer@gaia-g...,r-help@stat.math.ethz.ch,2007-07-06 10:43:12+00:00,Re: [R] RODBC problem,"Hello,\n\nas I wrote I call\n\n sqlFetch(chan...",0,1,"Re: [R] RODBC problem Hello,\n\nas I wrote I c...",Friday,6.0,July,2007.0,10:43:12,UTC,"Re: [R] RODBC problem Hello, as I wrote I call..."


In [113]:
df = df[df["year"] == 2007]

# Optional: reset index
df = df.reset_index(drop=True)
df

,sender,receiver,date,subject,body,label,urls,text,day_of_week,day,month,year,time,timezone,clean_text
0,Tomas Jacobs <RickyAmes@aol.com>,the00@speedy.uwaterloo.ca,2007-04-08 18:00:48+00:00,"Generic Cialis, branded quality@",\n\n\n\n\n\n\nDo you feel the pressure to perf...,1,0,"Generic Cialis, branded quality@ \n\n\n\n\n\n...",Sunday,8.0,April,2007.0,18:00:48,UTC,"Generic Cialis, branded quality@ Do you feel t..."
1,Yan Morin <yan.morin@savoirfairelinux.com>,debian-mirrors@lists.debian.org,2007-04-08 16:52:30+00:00,Typo in /debian/README,"Hi, i've just updated from the gulus and I che...",0,1,"Typo in /debian/README Hi, i've just updated f...",Sunday,8.0,April,2007.0,16:52:30,UTC,"Typo in /debian/README Hi, i've just updated f..."
2,Sheila Crenshaw <7stocknews@tractionmarketing....,the00@plg.uwaterloo.ca,2007-04-08 17:12:19+00:00,authentic viagra,Mega authenticV I A G R A $ DISCOUNT priceC...,1,1,authentic viagra Mega authenticV I A G R A ...,Sunday,8.0,April,2007.0,17:12:19,UTC,authentic viagra Mega authenticV I A G R A $ D...
3,Stormy Dempsey <vqucsmdfgvsg@ruraltek.com>,opt4@speedy.uwaterloo.ca,2007-04-08 18:15:47+00:00,Nice talking with ya,"\nHey Billy, \n\nit was really fun going out t...",1,1,"Nice talking with ya \nHey Billy, \n\nit was r...",Sunday,8.0,April,2007.0,18:15:47,UTC,"Nice talking with ya Hey Billy, it was really ..."
4,"""Christi T. Jernigan"" <dcube@totalink.net>",ktwarwic@speedy.uwaterloo.ca,2007-04-08 17:19:07+00:00,or trembling; stomach cramps; trouble in sleep...,"\nsystem"" of the home. It will have the capab...",1,0,or trembling; stomach cramps; trouble in sleep...,Sunday,8.0,April,2007.0,17:19:07,UTC,or trembling; stomach cramps; trouble in sleep...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52708,SCC <Gerry.Rossi4360@kinki-kids.com>,Deficient <deficient@flax9.uwaterloo.ca>,2007-07-06 10:53:36+00:00,Job: just for you.,\n\n\n\nWhile we may have high ...,1,1,Job: just for you. \n\n\n\nWhile we may ...,Friday,6.0,July,2007.0,10:53:36,UTC,Job: just for you. While we may have high expe...
52709,Sydney Car Centre <Merrill8783@168city.com>,Gnitpick <gnitpick@flax9.uwaterloo.ca>,2007-07-06 10:59:51+00:00,the reply for your request for a job place [le...,\n\n\n\nWhile we may have high ...,1,1,the reply for your request for a job place [le...,Friday,6.0,July,2007.0,10:59:51,UTC,the reply for your request for a job place [le...
52710,Philippe Grosjean <phgrosjean@sciviews.org>,Duncan Murdoch <murdoch@stats.uwo.ca>,2007-07-06 10:57:17+00:00,"Re: [R] Me again, about the horrible documenta...","For those who are interested, I just cook a li...",0,1,"Re: [R] Me again, about the horrible documenta...",Friday,6.0,July,2007.0,10:57:17,UTC,"Re: [R] Me again, about the horrible documenta..."
52711,Bernhard Wellhöfer <Bernhard.Wellhoefer@gaia-g...,r-help@stat.math.ethz.ch,2007-07-06 10:43:12+00:00,Re: [R] RODBC problem,"Hello,\n\nas I wrote I call\n\n sqlFetch(chan...",0,1,"Re: [R] RODBC problem Hello,\n\nas I wrote I c...",Friday,6.0,July,2007.0,10:43:12,UTC,"Re: [R] RODBC problem Hello, as I wrote I call..."
